# Semi-definite integrals with Gauss-Laguerre quadrature

For the multilayer finite dipole method (FDM), we need to calculate the electric potential and field induced at a surface by a charge $q$ using the integrals
$$
\begin{align*}
  \phi &= \int_0^\infty \beta(k) e^{-2 z_q k} dk, \text{ and}\\
  E_z &= \int_0^\infty \beta(k) k e^{-2 z_q k} dk,
\end{align*}
$$
where $k$ is the in-plane momentum, $\beta(k)$ is the effective surface reflection coefficient, and $z_q$ is the height of the charge above the surface.

`scipy.integrate.quad_vec` can perform infinite integrals, but it can't be compiled using `numba`, which means it can't be called from within other `numba`-compiled functions.
I want to find a way to evaluate the integrals without relying on `scipy.integrate.quad_vec`.

## Gauss-Laguerre quadrature

Gauss-Laguerre quadrature is a version of Gaussian quadrature which evaluates exponentially weighted integrals according to
$$
\int_0^{\infty} e^{-x} f(x) dx \approx \sum_{n=1}^N w_n f(x_n),
$$
where $x_n$ is the $n^{th}$ root of the Laguerre polynomial
$$
L_N(x) = \sum_{n=0}^{N} {N \choose n} \frac{(-1)^n}{n!} x^n,
$$
and $w_n$ is a weight given by
$$
w_n = \frac{x_n}{\left((N + 1) L_{N+1}(x_n) \right)^2}.
$$

Evaluating the roots and weights can be complicated, but thankfully numpy has a function `numpy.polynomial.laguerre.laggauss(deg)`, which returns `(x, y)`, the roots and weights of the Laguerre polynomial for $N=$ `deg`.

## Adapting the integrals

To convert the integrals to the needed form, we can use the substitution $x = 2 z_q k$, so that
$$
\begin{align*}
  \phi
  & = \frac{1}{2 z_q} \int_0^\infty \beta\left(\frac{x}{2 z_q}\right) e^{-x} dx, \text{ and}\\
  E_z
  & = \frac{1}{4 z_q^2} \int_0^\infty \beta\left(\frac{x}{2 z_q}\right) x e^{-x} dx.
\end{align*}
$$

We can then evaluate the integrals with the approximations
$$
\begin{align*}
  \phi
  & \approx \frac{1}{2 z_q} \sum_{n=1}^N w_n \beta\left(\frac{x_n}{2 z_q}\right), \text{ and}\\
  E_z
  & \approx \frac{1}{4 z_q^2} \sum_{n=1}^N w_n \beta\left(\frac{x_n}{2 z_q}\right) x_n.
\end{align*}
$$